In [179]:
import pandas as pd
import dask.dataframe as dd

In [180]:
train_df = dd.read_csv("dataset/train_ver2.csv", blocksize=10e6, dtype={'ind_nom_pens_ult1': 'float64', 'ind_nomina_ult1': 'float64', 
                                                                        'conyuemp': 'object', 'indrel_1mes': 'object'})

In [181]:
translation_df = pd.read_csv("feature_translation.csv")
translation_dict = translation_df.set_index('original').to_dict()['translation']
reverse_translation_dict = translation_df.set_index('translation').to_dict()['original']

In [182]:
train_df_tr = train_df.rename(columns=translation_dict)

In [183]:
train_df_tr.head()

,Row_Date,Customer_Code,Employee_Type,Country_Residence,Sex,Age,Registration_Date,Is_New_Customer,Customer_Seniority_Months,First_Or_Primary,...,FT_Mortgage,FT_Pensions1,FT_Loans,FT_Taxes,FT_Credit_Card,FT_Securities,FT_Home_Account,FT_Payroll,FT_Pensions2,FT_Direct_Debit
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [184]:
indexed_df = train_df_tr.set_index('Customer_Code')

/home/user279/anaconda3/lib/python3.7/site-packages/dask/core.py:118: DtypeWarning: Columns (5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


In [185]:
customers = indexed_df.index.unique().compute()

/home/user279/anaconda3/lib/python3.7/site-packages/dask/core.py:118: DtypeWarning: Columns (5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


In [186]:
len(customers)

956645

In [187]:
HOW_MANY_CUSTOMERS = 40000

In [188]:
partial_customers = set(customers.sample(HOW_MANY_CUSTOMERS, random_state=1).values)

In [189]:
partial_dataset = indexed_df.loc[indexed_df.index.isin(partial_customers)].compute()

/home/user279/anaconda3/lib/python3.7/site-packages/dask/core.py:118: DtypeWarning: Columns (5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


In [190]:
partial_dataset['Row_Date'] = pd.to_datetime(partial_dataset['Row_Date'])
partial_dataset['Last_Date_primary'] = pd.to_datetime(partial_dataset['Last_Date_primary'])
partial_dataset['Registration_Date'] = pd.to_datetime(partial_dataset['Registration_Date'])

In [191]:
partial_dataset['Age'] = pd.to_numeric(partial_dataset['Age'], errors='coerce')
partial_dataset['Customer_Seniority_Months'] = pd.to_numeric(partial_dataset['Customer_Seniority_Months'], errors='coerce')

In [192]:
partial_dataset['Is_Residence_Same_Bank'] = partial_dataset['Is_Residence_Same_Bank'].map({'S': 1, 'N': 0, 1: 1, 0: 0}).astype('float64')
partial_dataset['Is_Birth_Same_Bank'] = partial_dataset['Is_Birth_Same_Bank'].map({'S': 1, 'N': 0, 1: 1, 0: 0}).astype('float64')
partial_dataset['Is_Spouse_Of_Employee'] = partial_dataset['Is_Spouse_Of_Employee'].map({'S': 1, 'N': 0, 1: 1, 0: 0}).astype('float64')
partial_dataset['Is_Deceased'] = partial_dataset['Is_Deceased'].map({'S': 1, 'N': 0, 1: 1, 0: 0}).astype('float64')

In [193]:
categorical_fields = partial_dataset.dtypes[partial_dataset.dtypes == 'object'].index

for field in categorical_fields:
    partial_dataset[field] = partial_dataset[field].astype('category')

In [195]:
partial_dataset.to_pickle("dataset/partial_train.pkl")